In [1]:
# Dependencies and Setup
import pandas as pd

#check Pandas version 
#print(pd.__version__)

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data_complete_df.head()

In [2]:
#drop unneccessary information
school_cleaned_df = school_data_complete_df.drop(school_data_complete_df.columns[1:4], axis = 1)
#rename columns
school_cleaned_df = school_cleaned_df.rename(columns={"School ID": "school_id", "Student ID": "student_id"})
#school_cleaned_df.head()

In [3]:
#school_cleaned_df.dtypes

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
#calculate total number of schools
num_schools = school_cleaned_df["school_name"].value_counts()
#print(num_schools)

In [5]:
total_schools = num_schools.count()
#print(total_schools)

In [6]:
#Calculate total number of students
dist_students = len(school_cleaned_df)
#print(dist_students)

In [7]:
#Create dictionaries keyed on school id
budgets_dict = dict(zip(school_cleaned_df.school_id, school_cleaned_df.budget))
names_dict = dict(zip(school_cleaned_df.school_id, school_cleaned_df.school_name))
types_dict = dict(zip(school_cleaned_df.school_id, school_cleaned_df.type))
#print(budgets_dict)
#print(names_dict)
#print(types_dict)

In [8]:
#Calculate the total budget
budget_total = sum(budgets_dict.values())
#print(budget_total)

In [9]:
#Add total district math scores
dist_math = school_cleaned_df.loc[:,"math_score"].sum()
#print(dist_math)

In [10]:
#Calculate the average math score 
dist_math_avg = dist_math / dist_students
#print(dist_math_avg)

In [11]:
#Add total district reading scores
dist_reading = school_cleaned_df.loc[:,"reading_score"].sum()
#print(dist_reading)

In [12]:
#Calculate the average reading score
dist_reading_avg = dist_reading / dist_students
#print(dist_reading_avg)

In [13]:
#Calculate number of students w passing math score
dist_math_passing_df = school_cleaned_df.loc[school_cleaned_df["math_score"] >= 70]
#dist_math_passing_df.head()

dist_num_math = dist_math_passing_df["student_id"].count()
#print(dist_num_math)

In [14]:
#Calculate number of passing math students by school
#https://stackoverflow.com/questions/29876184/groupby-results-to-dictionary-of-lists
school_math_passing_dict = {k: list(v) for k, v in dist_math_passing_df.groupby("school_id")["student_id"]}
#print(school_math_passing_dict)
#len(school_math_passing)

In [15]:
#Calculate number of students w passing reading score
dist_reading_passing_df = school_cleaned_df.loc[school_cleaned_df["reading_score"] >= 70]
#dist_reading_passing_df.head()

dist_num_reading = dist_reading_passing_df["student_id"].count()
#print(dist_num_reading)

In [16]:
#Calculate number of passing reading students by school
school_reading_passing_dict = {k: list(v) for k, v in dist_reading_passing_df.groupby("school_id")["student_id"]}
#print(school_reading_passing_dict)
#len(school_reading_passing)

In [17]:
#Calculate Percentage of passing students
dist_math_percent = (dist_num_math / dist_students) * 100
dist_reading_percent = (dist_num_reading / dist_students) * 100
#print(dist_math_percent)
#print(dist_reading_percent)

In [18]:
#Calculate number of students passing both
#This feels inelegant; come back to this section
dist_passing_merged_df = pd.merge(dist_math_passing_df, dist_reading_passing_df, on="student_id", how="inner")
#dist_passing_merged_df.head()

In [19]:
dist_num_both = dist_passing_merged_df["student_id"].count()
#print(dist_num_both)

In [20]:
#Calculate the number of students passing both by school
school_passing_both_dict = {k: list(v) for k, v in dist_passing_merged_df.groupby("school_id_x")["student_id"]}
#print(school_passing_both_dict)

In [21]:
#Calculate percentage of dual-passing students
dist_both_percent = (dist_num_both / dist_students) * 100
#print(dist_both_percent)

In [22]:
#Build district summary data frame
district_summary_df = pd.DataFrame({"Total Schools" : [total_schools],
                                    "Total Students" : dist_students,
                                    "Total Budget" : budget_total,
                                    "Average Math Score" : dist_math_avg,
                                    "Average Reading Score" : dist_reading_avg,
                                    "% Passing Math" : dist_math_percent,
                                    "% Passing Reading" : dist_reading_percent,
                                    "% Overall Passing": dist_both_percent
                                   })
#Format entries : 
#https://stackoverflow.com/questions/32744997/python-pandas-apply-formatting-to-each-column-in-dataframe-using-a-dict-mapping
format_mapping = {"Total Students" : "{:,}", 
                  "Total Budget" : "${:,.2f}", 
                  "Average Math Score" : "{:.6f}", 
                  "Average Reading Score" : "{:.6f}", 
                  "% Passing Math" : "{:.6f}%",
                  "% Passing Reading" : "{:.6f}%",
                  "% Overall Passing" : "{:.6f}%"
                 }
#apply formatting
for key, value in format_mapping.items():
       district_summary_df[key] = district_summary_df[key].apply(value.format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853%,85.805463%,65.172326%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [57]:
#Group by school and count students per school
school_data_grouped = school_cleaned_df.groupby(school_cleaned_df.school_id)
school_students = school_data_grouped["size"].count()
school_students_list = list(school_students)
#print(school_students_list)

In [58]:
#Retrieve school names
school_name_list = list(names_dict.values())
#print(school_name_list)

In [59]:
#Retrieve school types
school_type_list = list(types_dict.values())
#print(school_type_list)

In [60]:
#Retrieve individual school budgets
school_budget_list = list(budgets_dict.values())
#print(school_budget_list)

In [61]:
#Calculate per-student budgets per school
per_student_budget = [b/s for b, s in zip(school_budget_list, school_students_list)]
#print(per_student_budget)

In [62]:
#calculate total math score by school
school_math = (school_data_grouped["math_score"].sum()) 
#print(school_math)

In [63]:
#Calculate average math score by school
school_math_avg = list(school_math / school_students)
#print(school_math_avg)

In [30]:
#Calculate total reading score by school
school_reading = school_data_grouped["reading_score"].sum()
#print(school_reading)

In [64]:
#Calcualte average reading score by school
school_reading_avg = list(school_reading / school_students)
#print(school_reading_avg)

In [65]:
#Calcualte number of passing math scores per school
#print(school_math_passing_dict)
school_math_passing = {k:len(v) for k, v in school_math_passing_dict.items()}        
#print(school_math_passing)

In [66]:
#Calculate percentage of students passing math per school
math_percent_list = list(school_math_passing.values())
#print(math_percent_list)
school_math_percent = list((math_percent_list / school_students) * 100)
#print(school_math_percent)

In [67]:
#Calculate number of passing reading scores per school
school_reading_passing = {k:len(v) for k, v in school_reading_passing_dict.items()}
#print(school_reading_passing)

In [68]:
#Calculate percentage of students passing reading per school
reading_percent_list = list(school_reading_passing.values())
#print(reading_percent_list)
school_reading_percent = list((reading_percent_list / school_students) * 100)
#print(school_reading_percent)

In [69]:
#Calculate number of passing students per school
school_passing_both = {k:len(v) for k, v in school_passing_both_dict.items()}
#print(school_passing_both)

In [70]:
#Calculate percentage of students passing math per school
both_percent_list = list(school_passing_both.values())
#print(both_percent_list)
school_both_percent = list((both_percent_list / school_students) * 100)
#print(school_both_percent)

In [38]:
school_summary_df = pd.DataFrame({"School Name" : school_name_list,
                                  "School Type" : school_type_list,
                                  "Total Students" : school_students_list,
                                  "Total School Budget" : school_budget_list,
                                  "Per Student Budget" : per_student_budget,
                                  "Average Math Score" : school_math_avg,
                                  "Average Reading Score" : school_reading_avg,
                                  "% Passing Math" : school_math_percent,
                                  "% Passing Reading" : school_reading_percent,
                                  "% Overall Passing" : school_both_percent})
#Sort alphabetically by school name
school_summary_df_sortabc = school_summary_df.sort_values("School Name")

#Format data frame
format_mapping_2 = {"Total Students" : "{:,}", 
                  "Total School Budget" : "${:,.2f}",
                  "Per Student Budget" : "${:,.2f}",
                  "Average Math Score" : "{:.6f}", 
                  "Average Reading Score" : "{:.6f}", 
                  "% Passing Math" : "{:.6f}%",
                  "% Passing Reading" : "{:.6f}%",
                  "% Overall Passing" : "{:.6f}%"
                  }
#apply formatting
for key, value in format_mapping_2.items():
       school_summary_df_sortabc[key] = school_summary_df_sortabc[key].apply(value.format)
school_summary_df_sortabc

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
7,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064%,81.933280%,54.642283%
6,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,91.334769%
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,53.204476%
13,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602%,79.299014%,54.289887%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,90.599455%
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967%,80.862999%,53.527508%
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855%,96.252927%,89.227166%
0,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922%,81.316421%,53.513884%
12,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551%,81.222432%,53.539172%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595%,95.945946%,90.540541%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [39]:
school_summary_sort_top_overall = school_summary_df.sort_values("% Overall Passing", ascending = False)
#apply formatting
for key, value in format_mapping_2.items():
       school_summary_sort_top_overall[key] = school_summary_sort_top_overall[key].apply(value.format)
school_summary_sort_top_overall.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,91.334769%
14,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171%,97.308869%,90.948012%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,90.599455%
5,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718%,96.539641%,90.582567%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595%,95.945946%,90.540541%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [40]:
school_summary_sort_bottom_overall = school_summary_df.sort_values("% Overall Passing", ascending = True)
#apply formatting
for key, value in format_mapping_2.items():
       school_summary_sort_bottom_overall[key] = school_summary_sort_bottom_overall[key].apply(value.format)
school_summary_sort_bottom_overall.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592%,80.220055%,52.988247%
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,53.204476%
0,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922%,81.316421%,53.513884%
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967%,80.862999%,53.527508%
12,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551%,81.222432%,53.539172%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [71]:
#Remind myself what the heck I started wtih
#school_data_complete_df.head()
#also, haha on student 3: https://www.scu.edu/mathcs/faculty-and-staff/richard-scott/
#PS - there's a typo in the instructions above - should read "Math Score"

In [72]:
#rename columns
school_grades_df = school_data_complete_df.rename(columns={"School ID": "school_id", "Student ID": "student_id"})
#school_grades_df.head()

In [73]:
#Sort by grade 9
school_grade9_m = school_grades_df.loc[school_grades_df["grade"] == "9th"]
#school_grade9_m
grade9m_grouped = school_grade9_m.groupby(school_grade9_m.school_id)
grade9m = list(grade9m_grouped["math_score"].mean())
#print(grade9m)

In [74]:
#Sort by grade 10
school_grade10_m = school_grades_df.loc[school_grades_df["grade"] == "10th"]
#school_grade10_m
grade10m_grouped = school_grade10_m.groupby(school_grade10_m.school_id)
grade10m = list(grade10m_grouped["math_score"].mean())
#print(grade10m)

In [75]:
#Sort by grade 11
school_grade11_m = school_grades_df.loc[school_grades_df["grade"] == "11th"]
#school_grade11_m
grade11m_grouped = school_grade11_m.groupby(school_grade11_m.school_id)
grade11m = list(grade11m_grouped["math_score"].mean())
#print(grade11m)

In [76]:
#Sort by grade 12
school_grade12_m = school_grades_df.loc[school_grades_df["grade"] == "12th"]
#school_grade12_m
grade12m_grouped = school_grade12_m.groupby(school_grade12_m.school_id)
grade12m = list(grade12m_grouped["math_score"].mean())
#print(grade12m)

In [77]:
math_by_grade_df = pd.DataFrame({"School Name" : school_name_list,
                                 "9th" : grade9m,
                                 "10th" : grade10m,
                                 "11th" : grade11m,
                                 "12th" : grade12m})
#Sort alphabetically by school name
math_by_grade_df_sortabc = math_by_grade_df.sort_values("School Name")
#Format data frame
format_mapping_3 = {"9th" : "{:.6f}", 
                  "10th" : "{:.6f}", 
                  "11th" : "{:.6f}%",
                  "12th" : "{:.6f}%",
                   }
#apply formatting
for key, value in format_mapping_3.items():
       math_by_grade_df_sortabc[key] = math_by_grade_df_sortabc[key].apply(value.format)
math_by_grade_df_sortabc
#Reset index to school name
math_by_grade_df_sortabc.set_index("School Name")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588%,76.492218%
Cabrera High School,83.094697,83.154506,82.765560%,83.277487%
Figueroa High School,76.403037,76.539974,76.884344%,77.151369%
Ford High School,77.361345,77.672316,76.918058%,76.179963%
Griffin High School,82.044010,84.229064,83.842105%,83.356164%
Hernandez High School,77.438495,77.337408,77.136029%,77.186567%
Holden High School,83.787402,83.429825,85.000000%,82.855422%
Huang High School,77.027251,75.908735,76.446602%,77.225641%
Johnson High School,77.187857,76.691117,77.491653%,76.863248%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [78]:
#Sort by grade 9
school_grade9_r = school_grades_df.loc[school_grades_df["grade"] == "9th"]
#school_grade9_r
grade9r_grouped = school_grade9_r.groupby(school_grade9_r.school_id)
grade9r = list(grade9r_grouped["reading_score"].mean())
#print(grade9r)

In [79]:
#Sort by grade 10
school_grade10_r = school_grades_df.loc[school_grades_df["grade"] == "10th"]
#school_grade10_r
grade10r_grouped = school_grade10_r.groupby(school_grade10_r.school_id)
grade10r = list(grade10r_grouped["reading_score"].mean())
#print(grade10r)

In [80]:
#Sort by grade 11
school_grade11_r = school_grades_df.loc[school_grades_df["grade"] == "11th"]
#school_grade11_r
grade11r_grouped = school_grade11_r.groupby(school_grade11_r.school_id)
grade11r = list(grade11r_grouped["reading_score"].mean())
#print(grade11r)

In [81]:
#Sort by grade 12
school_grade12_r = school_grades_df.loc[school_grades_df["grade"] == "12th"]
#school_grade12_r
grade12r_grouped = school_grade12_r.groupby(school_grade12_r.school_id)
grade12r = list(grade12r_grouped["reading_score"].mean())
#print(grade12r)

In [82]:
reading_by_grade_df = pd.DataFrame({"School Name" : school_name_list,
                                 "9th" : grade9r,
                                 "10th" : grade10r,
                                 "11th" : grade11r,
                                 "12th" : grade12r})
#Sort alphabetically by school name
reading_by_grade_df_sortabc = reading_by_grade_df.sort_values("School Name")
#apply formatting
for key, value in format_mapping_3.items():
       reading_by_grade_df_sortabc[key] = reading_by_grade_df_sortabc[key].apply(value.format)
reading_by_grade_df_sortabc
#Reset index to school name
reading_by_grade_df_sortabc.set_index("School Name")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643%,80.912451%
Cabrera High School,83.676136,84.253219,83.788382%,84.287958%
Figueroa High School,81.198598,81.408912,80.640339%,81.384863%
Ford High School,80.632653,81.262712,80.403642%,80.662338%
Griffin High School,83.369193,83.706897,84.288089%,84.013699%
Hernandez High School,80.866860,80.660147,81.396140%,80.857143%
Holden High School,83.677165,83.324561,83.815534%,84.698795%
Huang High School,81.290284,81.512386,81.417476%,80.305983%
Johnson High School,81.260714,80.773431,80.616027%,81.227564%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [91]:
school_sort_psb = school_summary_df.sort_values("Per Student Budget")
#school_sort_psb
budget_bins = [550, 590, 630, 645, 700]
bbins_names = ["$550 - $590", "$590 - $630", "$630 - $645", "$645 - $700"]
school_sort_psb_binned = pd.cut(school_sort_psb["Per Student Budget"], budget_bins, labels = bbins_names, include_lowest = True)
school_sort_psb_binned

5     $550 - $590
8     $550 - $590
6     $550 - $590
10    $550 - $590
2     $590 - $630
9     $590 - $630
4     $590 - $630
7     $590 - $630
11    $630 - $645
14    $630 - $645
1     $630 - $645
13    $630 - $645
12    $645 - $700
3     $645 - $700
0     $645 - $700
Name: Per Student Budget, dtype: category
Categories (4, object): [$550 - $590 < $590 - $630 < $630 - $645 < $645 - $700]

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type